In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [3]:
#加载数据
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [4]:
#分离X和y变量
y = train_data['label']
X = train_data.drop(columns = 'label')
print(train_data.shape)

(42000, 785)


In [5]:
#归一化
X = X/255.0
test_data = test_data/255.0

print("X:", X.shape)

X: (42000, 784)


In [6]:
# 特征缩放
from sklearn.preprocessing import scale
X_scaled = scale(X)

# 划分训练集和验证集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2 ,random_state = 10)

In [59]:
import numpy as np
import pandas as pd
#自行定义的线性SVM类
class LinearSVM:
    def __init__(self, learning_rate=0.01, num_epochs=1000, C=1.0):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.C = C

    def fit(self, X, y):
        X = X.values if isinstance(X, pd.DataFrame) else X
        y = y.values if isinstance(y, pd.Series) else y
        
        self.classes = np.unique(y)
        self.classifiers = {}
        
        # Train one classifier for each class
        for c in self.classes:
            y_binary = np.where(y == c, 1, -1)
            self.classifiers[c] = self._train_one_vs_rest(X, y_binary)

    def _train_one_vs_rest(self, X, y):
        w = np.zeros(X.shape[1])
        b = 0
        
        for epoch in range(self.num_epochs):
            for i, x in enumerate(X):
                if y[i] * (np.dot(x, w) - b) >= 1:
                    w -= self.learning_rate * (2 * self.C * w)
                else:
                    w -= self.learning_rate * (2 * self.C * w - np.dot(x, y[i]))
                    b -= self.learning_rate * y[i]
        
        return w, b

    def predict(self, X):
        X = X.values if isinstance(X, pd.DataFrame) else X
        predictions = []
        for x in X:
            scores = {c: np.dot(x, self.classifiers[c][0]) - self.classifiers[c][1] for c in self.classes}
            prediction = max(scores, key=scores.get)
            predictions.append(prediction)
        return np.array(predictions)


In [60]:
# 创建一个LinearSVM对象
svm = LinearSVM()

# 使用训练集进行训练
svm.fit(X_train, y_train)

# 使用验证集进行预测
y_pred = svm.predict(X_test)


In [64]:
# 精确度、分类报告和混淆矩阵
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report
print("accuracy:", metrics.accuracy_score(y_true=y_test, y_pred=y_pred), "\n")
print(classification_report(y_test, y_pred)) 
print(metrics.confusion_matrix(y_true=y_test, y_pred=y_pred))

accuracy: 0.6059523809523809 

              precision    recall  f1-score   support

           0       0.65      0.72      0.69       796
           1       0.85      0.70      0.77       946
           2       0.63      0.61      0.62       856
           3       0.80      0.24      0.37       856
           4       0.50      0.52      0.51       826
           5       0.41      0.61      0.49       773
           6       0.61      0.82      0.70       787
           7       0.79      0.72      0.75       869
           8       0.51      0.39      0.45       836
           9       0.55      0.72      0.62       855

    accuracy                           0.61      8400
   macro avg       0.63      0.61      0.60      8400
weighted avg       0.64      0.61      0.60      8400

[[577   0   2  11  13  36 138  10   3   6]
 [  0 666  93   0   6   3   2   1 174   1]
 [117  16 522   6  33   5  75  30  30  22]
 [ 64  20  52 207  10 393  26  19  40  25]
 [  4  10   6   4 426  11  99  14   8 

In [7]:
# 使用sklearn库自带的SVM函数进行训练
model_linear = SVC(kernel='rbf')
model_linear.fit(X_train, y_train)
y_pred = model_linear.predict(X_test)

In [8]:
# 精确度、分类报告和混淆矩阵
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report
print("accuracy:", metrics.accuracy_score(y_true=y_test, y_pred=y_pred), "\n")
print(classification_report(y_test, y_pred)) 
print(metrics.confusion_matrix(y_true=y_test, y_pred=y_pred))

accuracy: 0.960952380952381 

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       796
           1       0.98      0.98      0.98       946
           2       0.95      0.96      0.96       856
           3       0.95      0.95      0.95       856
           4       0.97      0.97      0.97       826
           5       0.97      0.96      0.96       773
           6       0.97      0.97      0.97       787
           7       0.91      0.96      0.94       869
           8       0.97      0.93      0.95       836
           9       0.96      0.94      0.95       855

    accuracy                           0.96      8400
   macro avg       0.96      0.96      0.96      8400
weighted avg       0.96      0.96      0.96      8400

[[782   0   2   1   0   1   4   3   3   0]
 [  0 931   5   3   2   0   1   2   2   0]
 [  4   2 824   4   3   1   2  12   4   0]
 [  1   1  13 814   0   7   1  14   5   0]
 [  1   1   5   0 799   2   5   5   1  